In [1]:
from azureml.core import Workspace
ws = Workspace(subscription_id="ab0fea7d-9046-495f-840d-19a0430c1d73",
               resource_group="aisejam",
               workspace_name="amlsejam")

In [2]:
import urllib.request
from azureml.core.model import Model

# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "model.onnx")

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

Registering model bidaf_onnx


In [3]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)

In [4]:
# from azureml.core.webservice import LocalWebservice

# deployment_config = LocalWebservice.deploy_configuration(port=6789)

from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [5]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

/var/folders/bf/hw443_ds36985hq1dr8c1nmm0000gn/T/ipykernel_25933/1673426657.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-09 11:21:59+09:00 Creating Container Registry if not exists..
2023-01-09 11:31:59+09:00 Registering the environment.
2023-01-09 11:32:00+09:00 Building image..
2023-01-09 11:42:14+09:00 Generating deployment configuration.
2023-01-09 11:42:14+09:00 Submitting deployment to compute..
2023-01-09 11:42:22+09:00 Checking the status of deployment myservice..
2023-01-09 11:45:05+09:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /